# Import libs

In [ ]:
import os
import json
import random
import pickle
import numpy as np
import pandas as pd
from itertools import product
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

import utils
from VisionModels import CustomCNN, CustomEfficientNetB3
from VisionDatasets import ContactDataset

%load_ext autoreload
%autoreload 2

# Pre-settings

In [ ]:
parameters_json = os.path.join(os.getcwd(), 'settings/parameters.json')
paths_json = os.path.join(os.getcwd(), 'settings/paths.json')

In [ ]:
with open(parameters_json, 'r') as json_file:
    params = json.load(json_file)

with open(paths_json, 'r') as json_file:
    paths = json.load(json_file)

In [ ]:
random_seed = params["random_seed"]

os.environ['PYTHONHASHSEED'] = str(random_seed)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_set = [1, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 21, 41, 98, 107, 143]
val_set = [2, 4, 9, 11, 16, 18, 42, 116]
test_set = [37, 68, 71, 23, 22, 24, 25, 26, 27, 28, 29,
            46, 47, 38, 95, 104, 101, 92, 110, 45, 125, 134]

In [ ]:
model_set = ['CustomCNN', 'EfficientNet']
label_set = ['GT', 'MTurk']
model_combinations = list(product(model_set, label_set))

# Import Data

In [ ]:
data = pd.read_csv(paths['silicone']['labels'], header=0)

In [ ]:
# filter the data rows by the index of sets
train_data = data[data['HPC_Path'].str.extract(
    r'imageset_(\d+)').astype(int).isin(train_set).any(axis=1)]
val_data = data[data['HPC_Path'].str.extract(
    r'imageset_(\d+)').astype(int).isin(val_set).any(axis=1)]

In [ ]:
# add data to Datasets
train_dataloader = {}
val_dataloader = {}
loss_fn = {}

for label_name in label_set:

    image_col = 'HPC_Path'
    
    if label_name == 'GT':
        label_col = 'GT'
    elif label_name == 'MTurk':
        label_col = 'MTurk_Label'

    train_dataset = ContactDataset(
        images=train_data[image_col].tolist(),
        labels=train_data[label_col].to_numpy(),
        coords=list(zip(
            train_data['x'].astype(int),
            train_data['y'].astype(int))),
        jitter=True)

    val_dataset = ContactDataset(
        images=val_data[image_col].tolist(),
        labels=val_data[label_col].to_numpy(),
        coords=list(zip(
            train_data['x'].astype(int),
            train_data['y'].astype(int))),
        jitter=False)

    # create DataLoader with existed Datasets
    train_dataloader[label_name] = DataLoader(
        dataset=train_dataset,
        batch_size=params['batch_size'],
        num_workers=(16 if os.cpu_count() > 16 else os.cpu_count()),
        pin_memory=True,
        shuffle=True)

    val_dataloader[label_name] = DataLoader(
        dataset=val_dataset,
        batch_size=params['batch_size'],
        num_workers=(16 if os.cpu_count() > 16 else os.cpu_count()),
        pin_memory=True,
        shuffle=True)
    
    weights = train_dataset.getWeights().to(device)
    loss_fn[label_name] = nn.CrossEntropyLoss(weight=weights)


# Training

In [ ]:
for model_name, label_name in model_combinations:
    # select the model
    if model_name == 'CustomCNN':
        model = CustomCNN()
    elif model_name == 'EfficientNet':
        model = CustomEfficientNetB3()

    # set up the optimizer (hyper-parameters)
    optimizer = optim.Adam(
        model.parameters(),
        lr=params[model_name]['learning_rate'],
        weight_decay=params[model_name]['weight_decay'])

    # train and retrieve the metrics
    utils.train(
        model=model,
        optimizer=optimizer,
        loss_fn=loss_fn[label_name],
        dataloader=train_dataloader[label_name],
        val_dataloader=val_dataloader[label_name],
        device=device,
        use_tqdm=True,
        epochs=params['epochs'])

    utils.save_metrics(model, model_name, label_name)
    utils.save_state_dict(model, model_name, label_name)

# Test

In [ ]:
model_name = 'CustomCNN'
label_name = 'MTurk'
model = CustomCNN()

# set up the optimizer (hyper-parameters)
optimizer = optim.Adam(
    model.parameters(),
    lr=params[model_name]['learning_rate'],
    weight_decay=params[model_name]['weight_decay'])

# train and retrieve the metrics
utils.train(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn[label_name],
    dataloader=train_dataloader[label_name],
    val_dataloader=val_dataloader[label_name],
    device=device,
    use_tqdm=True,
    epochs=10)

utils.save_metrics(model, model_name, label_name)
utils.save_state_dict(model, model_name, label_name)

In [ ]:
pred, gt = utils.predict(
    model=model,
    dataloader=val_dataloader[label_name],
    device=device)

In [ ]:
binary_predictions = (pred > 0.5).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(gt, binary_predictions)

In [ ]:
from PIL import Image, ImageDraw
import torchvision
width, height = 936, 702
merged_image = Image.new('RGB', (width, height))

draw = ImageDraw.Draw(merged_image)

small_image_width, small_image_height = 234, 234
x, y = 0, 0

for images, labels in val_dataloader:
    for image in images:
        image = torchvision.transforms.ToPILImage()(image)
        merged_image.paste(image, (x, y))
    
        x += small_image_width
        
        if x + small_image_width > width:
            x = 0
            y += small_image_height

merged_image.show()